In [1]:
import numpy as np
import time
import os
from scipy.sparse import (
    identity as sparse_identity,
    diags as sparse_diags,
    kron as sparse_kron,
    csc_matrix,
    eye as sparse_eye
)
from scipy.sparse.linalg import expm as sparse_expm
from scipy.integrate import solve_ivp, RK45
from scipy.integrate._ivp.base import OdeSolver
from functools import reduce # Needed for sparse.kron reduction
import sys
from pathlib import Path
# Add the project root directory to Python path
notebook_path = Path().absolute()  # Gets the current notebook directory
project_root = notebook_path.parent  # Goes up one level to project root
sys.path.append(str(project_root))
from utils.funcs import load_params
import matplotlib.pyplot as plt

In [2]:

def sparse_destroy(dim):
    """Creates a sparse destruction operator."""
    if dim <= 0:
        raise ValueError("Dimension must be > 0")
    if dim == 1:
        return csc_matrix((1, 1), dtype=np.complex128)
    data = np.sqrt(np.arange(1, dim, dtype=np.complex128))
    offsets = [1]
    # FIX: Wrap 'data' in a list to match the list 'offsets'
    return sparse_diags([data], offsets, shape=(dim, dim), format='csc')

def sparse_num(dim):
    """Creates a sparse number operator."""
    if dim <= 0:
        raise ValueError("Dimension must be > 0")
    if dim == 1:
        return csc_matrix((1, 1), dtype=np.complex128)
    data = np.arange(dim, dtype=np.complex128)
    offsets = [0]
    # FIX: Wrap 'data' in a list to match the list 'offsets'
    return sparse_diags([data], offsets, shape=(dim, dim), format='csc')

def get_sparse_op(op, site, dims):
    """
    Creates a full-system sparse operator from a local operator
    using sparse.kron.
    """
    op_list = [sparse_identity(d, dtype=np.complex128, format='csc') for d in dims]
    op_list[site] = op
    return reduce(sparse_kron, op_list)

def propagator_ode_real(t, y_real, H0, H1, W, nu_delta, D):
    """
    The ODE function for the propagator U(t), compatible with solve_ivp.
    It evolves dU/dt = -i * H(t) * U(t)
    
    y_real = [U_flat_real, U_flat_imag] (size 2 * D*D)
    """
    D_sq = D * D
    # Reconstruct the complex flattened vector
    U_flat_complex = y_real[:D_sq] + 1j * y_real[D_sq:]
    
    # Reshape into a D x D matrix
    # We use 'F' (Fortran) order to match the flattening,
    # which is standard for quantum mechanics state vectors vs operators.
    U = U_flat_complex.reshape((D, D), order='F')
    
    # Calculate H(t)
    H_t = H0 + H1 * (W * np.cos(nu_delta * t))
    
    # Calculate dU/dt = -i * H(t) * U
    # H_t is (D, D), U is (D, D). Use matrix multiplication
    dU_dt_complex = -1j * (H_t @ U)
    
    # Flatten the complex derivative
    dU_dt_flat = dU_dt_complex.flatten(order='F')
    
    # Split back into real and imaginary parts
    return np.concatenate([dU_dt_flat.real, dU_dt_flat.imag])

def build_H(params):

    N = int(params['N'])
    n_max_transmon = int(params['n_max_transmon'])
    n_max_resonator = int(params['n_max_resonator'])

    eta = float(params['eta'])
    phiq = float(params['phiq'])
    phia = eval(str(params['phia']))
    J = eval(str(params['J']))
    nu = eval(str(params['nu']))
    delta = eval(str(params['delta']))
    de = float(params['de']) * delta
    wq = eval(str(params['wq']))
    EJ = eval(str(params['EJ']))

    # --- System Definition ---
    dims = [n_max_transmon if i % 2 == 0 else n_max_resonator for i in range(2 * N)]
    D_total = np.prod(dims)

    En = []
    for i in range(N):
        En.append(wq + i * de)
        En.append(nu)
    En = np.array(En)

    # --- Local Operators (Sparse) ---

    a_t = sparse_destroy(n_max_transmon)
    a_t_dag = a_t.conj().T.tocsc() 
    n_t = sparse_num(n_max_transmon)
    x_t = a_t_dag + a_t           

    a_r = sparse_destroy(n_max_resonator)
    a_r_dag = a_r.conj().T.tocsc() 
    n_r = sparse_num(n_max_resonator)
    x_r = a_r_dag + a_r           

    print(f"Total Hilbert space dimension: {D_total}")

    # --- Build Hamiltonian (Sparse) ---
    print("Building sparse H0...")
    # On-site energies
    H0 = csc_matrix((D_total, D_total), dtype=np.complex128)
    for i in range(2 * N):
        if i % 2 == 0:  # Transmon site
            H0 += En[i] * get_sparse_op(n_t, i, dims)
        else:  # Resonator site
            H0 += En[i] * get_sparse_op(n_r, i, dims)

    # EJ term: -0.5*EJ*(phi^2 + cos(phi))
    phi_squared_sum = csc_matrix((D_total, D_total), dtype=np.complex128)
    cos_phi_sum = csc_matrix((D_total, D_total), dtype=np.complex128)

    for i in range(N):
        print(f"Building phi term for site {i}...")
        # phi_op for the i-th Transmon-Resonator pair
        phi_op = (
            phiq * get_sparse_op(x_t, 2 * i, dims) + 
            phia * get_sparse_op(x_r, 2 * i + 1, dims)
        )
        
        # Contribution to the sum of phi^2 terms
        phi_squared_sum += phi_op * phi_op
        
        # Contribution to the sum of cos(phi) terms
        # This is the computationally heavy step
        print(f"Calculating matrix exponential for site {i}...")
        U_phi = sparse_expm(-1j * phi_op)
        cos_phi_sum += 0.5 * (U_phi + U_phi.conj().T)
        print(f"Done with site {i}.")

    H0 += EJ * (0.5 * phi_squared_sum + cos_phi_sum)

    # J coupling term
    for i in range(N - 1):
        H0 += J * get_sparse_op(x_t, 2 * i, dims) * get_sparse_op(x_t, 2 * i + 2, dims)

    print("Building sparse H1...")
    # Time-dependent part of Hamiltonian
    H1 = csc_matrix((D_total, D_total), dtype=np.complex128)
    for i in range(N):
        H1 += get_sparse_op(x_t, 2 * i, dims)

    return H0, H1

def initial_state(params):

    N = int(params['N'])
    n_max_transmon = int(params['n_max_transmon'])
    n_max_resonator = int(params['n_max_resonator'])

    # --- System Definition ---
    dims = [n_max_transmon if i % 2 == 0 else n_max_resonator for i in range(2 * N)]

    # --- Build Initial State (NumPy) ---
    psi_list = []
    for i in range(2 * N):
        dim_i = dims[i] # Get the correct dimension for the site
        state_vec = np.zeros(dim_i, dtype=np.complex128)
        if i == 0:
            state_vec[1] = 1.0 # Transmon 1 is |1>
        else:
            state_vec[0] = 1.0 # All others are |0>
        psi_list.append(state_vec)

    # Use reduce with np.kron for the state vector
    psi0_vec = reduce(np.kron, psi_list)

    return psi0_vec

def get_number_ops(params):
    
    N = int(params['N'])
    n_max_transmon = int(params['n_max_transmon'])
    n_max_resonator = int(params['n_max_resonator'])

    # --- System Definition ---
    dims = [n_max_transmon if i % 2 == 0 else n_max_resonator for i in range(2 * N)]

    n_t = sparse_num(n_max_transmon)
    n_r = sparse_num(n_max_resonator)

    # --- Build Observables (Sparse) ---
    number_ops = []
    for i in range(2 * N):
        if i % 2 == 0:
            number_ops.append(get_sparse_op(n_t, i, dims))
        else:
            number_ops.append(get_sparse_op(n_r, i, dims))

    return number_ops

def plot_sim(njt, tlist):
    # --- Create Plot ---
    fig, ax = plt.subplots(figsize=(12, 6))

    for i in range(njt.shape[1]):
        if i % 2 == 0:  # Transmon site
            label_base = f'Transmon_{i//2 + 1}'
            plot_label = f'<n> Transmon {i//2 + 1}'
        else:  # Resonator site
            label_base = f'Resonator_{i//2 + 1}'
            plot_label = f'<n> Resonator {i//2 + 1}'

        expect_values = njt[:,i]
        ax.plot(tlist, expect_values, label=plot_label)

    # --- Configure and Show Plot ---
    ax.set_xlabel("Time (ns)")
    ax.set_ylabel("Population")
    ax.legend(loc='upper left')
    ax.grid(True)
    plt.show()

### Set $E_{n}\$ to be the inital state Using wrong number op